# import libraries

In [70]:
import pandas as pd

import string

import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer,PorterStemmer

from nltk import pos_tag

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# import the data

In [23]:
data= pd.read_csv("Restaurant_Reviews.csv")
data.head(20)

,Review,Liked
0,Wow... Loved this place.,Yes
1,Crust is not good.,No
2,Not tasty and the texture was just nasty.,No
3,Stopped by during the late May bank holiday of...,Yes
4,The selection on the menu was great and so wer...,Yes
5,Now I am getting angry and I want my damn pho.,No
6,Honeslty it didn't taste THAT fresh.,No
7,The potatoes were like rubber and you could te...,No
8,The fries were great too.,Yes
9,A great touch.,Yes


In [24]:
data.shape

(2220, 2)

In [26]:
data.describe().transpose()

,count,unique,top,freq
Review,2220,1610,The restaurant had clean and well-maintained f...,31
Liked,2220,2,No,1119


In [71]:
count= data.isnull().sum().sort_values(ascending=False)
percentage= ((data.isnull().sum()/len(data)*100)).sort_values(ascending=False)
missing_data=pd.concat([count,percentage],axis=1,keys=['count,percentage'])
missing_data

,"count,percentage"
Review,0
Liked,0


# Preprocessing

In [60]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    # original of the words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # remove stop words
    tokens = [word for word in tokens if word.lower() not in stop_words]
    # Remove punctuation
    tokens = [word for word in tokens if word not in string.punctuation]

    return ' '.join(tokens)

data['Review'] = data['Review'].apply(preprocess_text)

0                                          wow love place
1                                              crust good
2                                      tasti textur nasti
3       stop late may bank holiday rick steve recommen...
4                                 select menu great price
                              ...                        
2215                        staff untrain unfamiliar menu
2216                         portion skimpi left u hungri
2217                 restaur unclean restroom unplea odor
2218                             waiter slow forget order
2219                          food poorli cook tast burnt
Name: Review, Length: 2220, dtype: object

# Feature Extraction (Bag of Words)

In [67]:
# convert the data to vectors of the word and the number of his appearence
vectorizer = CountVectorizer()

# feature column
x = vectorizer.fit_transform(data['Review'])

# target column
y = data['Liked']

# choose model and train it

In [68]:
# spilt the data to train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

# train the data by using RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=0)
classifier.fit(X_train, y_train)

# prediction
y_pred = classifier.predict(X_test)


# evaluate the model

In [80]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='Yes')
recall = recall_score(y_test, y_pred, pos_label='Yes')
f1 = f1_score(y_test, y_pred, pos_label='Yes')
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy*100:.2f} %')
print(f'Precision: {precision*100:.2f} %')
print(f'Recall: {recall*100:.2f} %')
print(f'F1 Score: {f1*100:.2f} %')
print(f'Classification Report:\n{classification_rep}')


Accuracy: 88.29 %
Precision: 91.04 %
Recall: 84.33 %
F1 Score: 87.56 %
Classification Report:
              precision    recall  f1-score   support

          No       0.86      0.92      0.89       227
         Yes       0.91      0.84      0.88       217

    accuracy                           0.88       444
   macro avg       0.89      0.88      0.88       444
weighted avg       0.88      0.88      0.88       444

